# Transformar um arquivo JSON em CSV

Dataset: Paper Reviews
http://archive.ics.uci.edu/ml/datasets/Paper+Reviews

Os dados estão no formato JSON (com objetos aninhados), mas para facilitar o processamento, é melhor converter para uma estrutura de tabela (DataFrame).

In [21]:
import json
from itertools import chain

import pandas as pd

In [15]:
with open('data/reviews.json', 'r') as f:
    content = f.read()

papers = json.loads(content)['paper']

In [16]:
# ver a estrutura da primeira amostra
papers[0]

{'id': 1,
 'preliminary_decision': 'accept',
 'review': [{'confidence': '4',
   'evaluation': '1',
   'id': 1,
   'lan': 'es',
   'orientation': '0',
   'remarks': '',
   'text': '- El artículo aborda un problema contingente y muy relevante, e incluye tanto un diagnóstico nacional de uso de buenas prácticas como una solución (buenas prácticas concretas). - El lenguaje es adecuado.  - El artículo se siente como la concatenación de tres artículos diferentes: (1) resultados de una encuesta, (2) buenas prácticas de seguridad, (3) incorporación de buenas prácticas. - El orden de las secciones sería mejor si refleja este orden (la versión revisada es #2, #1, #3). - El artículo no tiene validación de ningún tipo, ni siquiera por evaluación de expertos.',
   'timespan': '2010-07-05'},
  {'confidence': '4',
   'evaluation': '1',
   'id': 2,
   'lan': 'es',
   'orientation': '1',
   'remarks': '',
   'text': 'El artículo presenta recomendaciones prácticas para el desarrollo de software seguro. S

Para simplificar, iremos ignorar os valores em `remarks`, `text`, `lan`.
As demais informações serão mantidas.

In [34]:
def map_paper_to_reviews(paper):
    for review in paper['review']:
        yield {
            'paper_id': paper['id'],
            'preliminary_decision': paper['preliminary_decision'],
            'review_id': review['id'],
            'confidence': int(review['confidence']) if review['confidence'] else None,
            'evaluation': int(review['evaluation']) if review['evaluation'] else None,
            'orientation': int(review['orientation']) if review['orientation'] else None,
            'timespan': review['timespan']
        }
    
reviews = map(map_paper_to_reviews, papers)
reviews = chain.from_iterable(reviews)
reviews = list(reviews)
reviews[:5]  # mostra os 5 primeiros elementos

[{'paper_id': 1,
  'preliminary_decision': 'accept',
  'review_id': 1,
  'confidence': 4,
  'evaluation': 1,
  'orientation': 0,
  'timespan': '2010-07-05'},
 {'paper_id': 1,
  'preliminary_decision': 'accept',
  'review_id': 2,
  'confidence': 4,
  'evaluation': 1,
  'orientation': 1,
  'timespan': '2010-07-05'},
 {'paper_id': 1,
  'preliminary_decision': 'accept',
  'review_id': 3,
  'confidence': 5,
  'evaluation': 1,
  'orientation': 1,
  'timespan': '2010-07-05'},
 {'paper_id': 2,
  'preliminary_decision': 'accept',
  'review_id': 1,
  'confidence': 4,
  'evaluation': 2,
  'orientation': 1,
  'timespan': '2010-07-05'},
 {'paper_id': 2,
  'preliminary_decision': 'accept',
  'review_id': 2,
  'confidence': 4,
  'evaluation': 2,
  'orientation': 0,
  'timespan': '2010-07-05'}]

In [70]:
# dados organizados por review
df = pd.DataFrame(reviews)
df.head()

,paper_id,preliminary_decision,review_id,confidence,evaluation,orientation,timespan
0,1,accept,1,4.0,1,0,2010-07-05
1,1,accept,2,4.0,1,1,2010-07-05
2,1,accept,3,5.0,1,1,2010-07-05
3,2,accept,1,4.0,2,1,2010-07-05
4,2,accept,2,4.0,2,0,2010-07-05


In [71]:
df.describe()

,paper_id,review_id,confidence,evaluation,orientation
count,405.000000,405.000000,403.000000,405.000000,405.000000
mean,84.945679,1.824691,3.573201,0.182716,-0.212346
std,49.854958,0.821362,0.844341,1.502868,1.019292
min,1.000000,1.000000,1.000000,-2.000000,-2.000000
25%,38.000000,1.000000,3.000000,-1.000000,-1.000000
50%,92.000000,2.000000,4.000000,0.000000,0.000000
75%,126.000000,2.000000,4.000000,2.000000,1.000000
max,172.000000,4.000000,5.000000,2.000000,2.000000


Se o objetivo é fazer uma análise dos reviews, os dados já estão no formato esperado.
Cada linha representa um registro (review) e cada coluna é uma variável.

Exemplos de perguntas que podem ser solucianadas com esse formato:

* Qual a avaliação média do primeiro review de cada paper?
* Quantas avaliações cada paper tem em média?
* Existe algum paper aceito, com review ruim?

Exemplos de perguntas que precisam dos dados em outro formato:

* Qual a contagem de papers aceitos/rejeitados?
* Qual a avaliação média de cada paper?

Note que nesse último exemplo, as perguntas são relacionadas aos papers, e não aos reviews diretamente.
Entretanto, os dados estão organizados de forma que cada linha da tabela representa um review e não um paper.
Assim, as informações de um paper estão distribuídas em várias linhas.

Para poder responder as perguntas acima, é necessário agrupar os reviews como colunas de cada linha.
A primeira parte desse processo é descobrir qual o máximo de reviews por paper.

In [72]:
# máximo de reviews por paper
df.groupby('paper_id').review_id.count().max()

4

In [84]:
# Referências https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
papers_df = df.pivot_table(
    index=['paper_id', 'preliminary_decision'],
    columns='review_id',
    values=['confidence', 'evaluation', 'orientation']
)
papers_df

confidence               evaluation            \
review_id                              1    2    3   4          1    2    3   
paper_id preliminary_decision                                                 
1        accept                      4.0  4.0  5.0 NaN        1.0  1.0  1.0   
2        accept                      4.0  4.0  4.0 NaN        2.0  2.0  2.0   
3        accept                      4.0  3.0  3.0 NaN        2.0  2.0  0.0   
4        accept                      4.0  2.0  NaN NaN        2.0 -2.0  NaN   
5        accept                      4.0  4.0  5.0 NaN        2.0  2.0  2.0   
...                                  ...  ...  ...  ..        ...  ...  ...   
168      accept                      3.0  4.0  NaN NaN        2.0 -2.0  NaN   
169      accept                      3.0  4.0  NaN NaN        1.0  1.0  NaN   
170      reject                      5.0  4.0  NaN NaN       -1.0 -1.0  NaN   
171      accept                      4.0  3.0  NaN NaN       -1.0  1.0  NaN   
172      no decision                 4.0  NaN  NaN NaN        1.0  NaN  NaN   

                                  orientation                
review_id                       4           1    2    3   4  
paper_id preliminary_decision                                
1        accept               NaN         0.0  1.0  1.0 NaN  
2        accept               NaN         1.0  0.0  0.0 NaN  
3        accept               NaN         1.0  1.0 -1.0 NaN  
4        accept               NaN         2.0 -1.0  NaN NaN  
5        accept               NaN         1.0  0.0  1.0 NaN  
...                            ..         ...  ...  ...  ..  
168      accept               NaN         1.0  0.0  NaN NaN  
169      accept               NaN         0.0  0.0  NaN NaN  
170      reject               NaN        -2.0 -1.0  NaN NaN  
171      accept               NaN         0.0 -1.0  NaN NaN  
172      no decision          NaN        -1.0  NaN  NaN NaN  

[169 rows x 12 columns]

In [85]:
#papers_df.columns = papers_df.columns.to_flat_index()
papers_df.columns =['_'.join(map(str, v)) for v in papers_df.columns]
papers_df = papers_df.reset_index()
papers_df

,paper_id,preliminary_decision,confidence_1,confidence_2,confidence_3,confidence_4,evaluation_1,evaluation_2,evaluation_3,evaluation_4,orientation_1,orientation_2,orientation_3,orientation_4
0,1,accept,4.0,4.0,5.0,NaN,1.0,1.0,1.0,NaN,0.0,1.0,1.0,NaN
1,2,accept,4.0,4.0,4.0,NaN,2.0,2.0,2.0,NaN,1.0,0.0,0.0,NaN
2,3,accept,4.0,3.0,3.0,NaN,2.0,2.0,0.0,NaN,1.0,1.0,-1.0,NaN
3,4,accept,4.0,2.0,NaN,NaN,2.0,-2.0,NaN,NaN,2.0,-1.0,NaN,NaN
4,5,accept,4.0,4.0,5.0,NaN,2.0,2.0,2.0,NaN,1.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,168,accept,3.0,4.0,NaN,NaN,2.0,-2.0,NaN,NaN,1.0,0.0,NaN,NaN
165,169,accept,3.0,4.0,NaN,NaN,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
166,170,reject,5.0,4.0,NaN,NaN,-1.0,-1.0,NaN,NaN,-2.0,-1.0,NaN,NaN
167,171,accept,4.0,3.0,NaN,NaN,-1.0,1.0,NaN,NaN,0.0,-1.0,NaN,NaN
